<a href="https://colab.research.google.com/github/lithops-zty/SSEF-RO021/blob/main/multivariate_model_encoder_decoder_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import array
import pandas as pd
from pandas import Series

import datetime
from datetime import datetime
from datetime import timedelta

import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense,TimeDistributed, Flatten, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import MinMaxScaler
from keras.regularizers import l2

from os.path import join

from functools import reduce

import keras

from matplotlib import pyplot as plt
from matplotlib.font_manager import FontProperties

from os.path import join


## Variables

In [ ]:
train_year = 2019
# test_year = 2021
hour_bins = 3
hour_resample = str(hour_bins) + "H"



n_steps = 15
n_input = 20
n_output = 7
rolling = False # if True, include Return and Borrow in X.

train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 1 - train_ratio - valid_ratio

include_covid = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_datapath = f'/content/drive/MyDrive/Colab Notebooks/Toronto_data/data_{train_year}/{train_year}_grouped_{hour_bins}hr_multivariate.csv'
# test_datapath = f'/content/drive/MyDrive/Colab Notebooks/Toronto_data/data_{test_year}/{test_year}_grouped_{hour_bins}hr_multivariate.csv'

# Data Preparation

In [ ]:
def data_prep(path):

  with open(path, 'r', encoding='utf8') as f:
    dataframe = pd.read_csv(f)
    dataframe['Time'] = pd.to_datetime(dataframe["Time"], errors = "coerce")
    dataframe = dataframe.set_index(['Station Id', 'Time'])


  fields = ["Borrow", "Return", "tempC", "FeelsLikeC", "IsHoliday", "Hour", "IsWeekday", "Month", "Latitude", "Longitude"]
  if include_covid:
    fields.extend(['new_cases', 'new_deaths'])
    
  dataframe = dataframe[fields]  

  dataframe['sin_time'] = np.sin(2*np.pi*dataframe.Hour/24)
  dataframe['cos_time'] = np.cos(2*np.pi*dataframe.Hour/24)

  dataframe.drop('Hour', axis=1, inplace=True)

  dataframe['sin_month'] = np.sin(2*np.pi*dataframe.Month/12)
  dataframe['cos_month'] = np.cos(2*np.pi*dataframe.Month/12)

  dataframe.drop('Month', axis=1, inplace=True)

  return dataframe

df = data_prep(train_datapath)

df

In [ ]:
def scale_data(df):
  columns = ['Borrow', 'Return', 'tempC', 'FeelsLikeC']
  if include_covid:
    columns.extend(['new_cases', 'new_deaths'])
  df_scaled = df.copy()
  scaling_factor = pd.Series(index=columns, dtype=float)
  for c in scaling_factor.index:
    negative_max = abs(df_scaled[c].min())
    positive_max = df_scaled[c].max()
    scaling_factor[c] = max(negative_max, positive_max)
    if scaling_factor[c] != 0: # skip only when all values in that column == 0
      df_scaled[c] = df_scaled[c] / scaling_factor[c]
  return df_scaled, scaling_factor

def normalise_data(df):
  columns = ['Latitude', 'Longitude']
  df[columns] = (df[columns]-df[columns].min())/(df[columns].max()-df[columns].min())
  return df

df_scaled, scaling_factor = scale_data(df)

df_scaled = normalise_data(df_scaled)

df_scaled

In [ ]:
scaling_factor

In [ ]:
def df23d(df): # (lvl 0 idx, lvl 1 idx, columns)
  E = df.groupby(level=0).apply(pd.DataFrame.to_numpy)
  return np.stack(E.values)

p = array([[[2,2], [3,3], 
      [4,4], [5,5], 
      [6,6],[7,7], 
      [8,8],[9,9]]])



n = 2
def to_series(df):
  def work(df2):
    arr = df23d(df2)
    arr = np.squeeze(arr)
    return np.stack([arr[x-n_input-n_output:x] for x in range(n_input+n_output, arr.shape[0], n_steps)])

  return np.concatenate(df.groupby(level=0).apply(work).values)
    

all_data = to_series(df_scaled)


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_valid_data = train_test_split(all_data, train_size=train_ratio, random_state=20211207)
validation_data, test_data = train_test_split(test_valid_data, train_size=valid_ratio/(valid_ratio+test_ratio), random_state=20211207)


In [ ]:
train_data[:,:20,:2].shape

In [ ]:
def split_xy(data):
  return (data[:, :n_input] if rolling else data[:, :n_input, 2:]), data[:, n_input:, :2]

In [ ]:

train_X, train_Y = split_xy(train_data)
val_X, val_Y = split_xy(validation_data)
test_X, test_Y = split_xy(test_data)

train_X.shape, train_Y.shape, val_X.shape, val_Y.shape, test_X.shape, test_Y.shape


In [ ]:
test_Y[459]

# Build & Train & Test Model

In [ ]:
model_name = f'{train_year}_{hour_bins}hr_'
if rolling:
  model_name += 'rolling_'
if include_covid:
  model_name += 'include_covid_'
model_name += 'best_model'

In [ ]:
def init_model():
  n_in_features, n_out_features = train_X.shape[2], train_Y.shape[2]
	
  model = Sequential()
  model.add(LSTM(00, activation='relu', input_shape=(n_input, n_in_features), dropout = 0.2))
  model.add(RepeatVector(n_output))
  model.add(LSTM(00, activation='relu', return_sequences=True, dropout = 0.2))
  model.add(TimeDistributed(Dense(50, activation='relu')))
  model.add(TimeDistributed(Dense(n_out_features, activation="relu")))
  model.compile(loss='mse', optimizer='adam')

  model.summary()
  
  return model


def fit_model(model):
  epochs, batch_size = 25, 128
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

  mc = ModelCheckpoint(f'/content/drive/MyDrive/Colab Notebooks/Toronto_data/models/{model_name}', monitor='val_loss')

  history = model.fit(train_X, train_Y, 
                      epochs=epochs, 
                      batch_size=batch_size, 
                      validation_data=(val_X, val_Y), 
                      callbacks = [mc,es])

  return history

In [ ]:
model = init_model()

In [ ]:
# model = tf.keras.models.load_model(f'/content/drive/MyDrive/Colab Notebooks/Toronto_data/models/{model_name}')

In [ ]:
history = fit_model(model)

In [ ]:
# save_path = f'/content/drive/MyDrive/Colab Notebooks/Toronto_data/models/{train_year}_{hour_bins}hr_{"rolling_" if rolling else ""}best_model'
# model.save(save_path)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.savefig("Model_loss_dropout.png")



In [ ]:
model.evaluate(test_X, test_Y)

In [ ]:
def predict(stn_id, pred_start):
  pred_start = pd.Timestamp(pred_start)
  X_end = pred_start - pd.Timedelta(hours=1 * hour_bins)
  X_start = X_end - pd.Timedelta(hours=(n_input-1) * hour_bins)

  X = df_scaled.loc[(stn_id, X_start):(stn_id, X_end)]
  arr_X = X.iloc[:,(0 if rolling else 2):].to_numpy()[None,:,:]

  pred_interval = pd.date_range(pred_start, pred_start + pd.Timedelta(hours=(n_output-1) * hour_bins), freq=hour_resample)
  pred = model.predict(arr_X)[0]
  pred[:,0] = pred[:,0] * scaling_factor['Borrow']
  pred[:,1] = pred[:,1] * scaling_factor['Return']

  return pd.DataFrame({'Borrow':pred[:,0], 'Return':pred[:,1]},index=pred_interval)

predict(7000, f'{train_year}/2/1')

In [ ]:
def truth(stn_id, start_time, end_time):      
  start_time = pd.Timestamp(start_time)
  end_time = pd.Timestamp(end_time)
  return df[['Borrow', 'Return']].loc[stn_id].loc[start_time:end_time]

truth(7019, f'{train_year}/1/2', f'{train_year}/1/3 00:00:00')

In [ ]:
from random import randrange, choice
def plot_pred(stn_id, start_time):
  fig, (ax0, ax1) = plt.subplots(1,2, figsize=(18,5))

  df_X = truth(stn_id, pd.Timestamp(start_time)-pd.Timedelta(hours=n_input*hour_bins), start_time - pd.Timedelta(hours=1*hour_bins))
  df_truth = pd.concat([df_X.iloc[-1:], truth(stn_id, start_time, start_time + pd.Timedelta(hours=(n_output-1)*hour_bins))], axis=0)
  df_pred = pd.concat([df_X.iloc[-1:], predict(stn_id, start_time)], axis=0).round()

  df_truth['Borrow'].plot(ax=ax0, label='truth')
  df_pred['Borrow'].plot(ax=ax0, label='predicted')
  df_X['Borrow'].plot(ax=ax0, label='input')
  
  df_truth['Return'].plot(ax=ax1, label='truth')
  df_pred['Return'].plot(ax=ax1, label='predicted')
  df_X['Return'].plot(ax=ax1, label='input')

  ax0.set_title(f'Bike Borrowed: Station ID={stn_id}')
  ax1.set_title(f'Bike Returned: Station ID={stn_id}')
  ax0.legend()
  ax1.legend()



num = randrange(7004, 7500)
time = choice(pd.date_range(f'{train_year}/2/3', f'{train_year}/12/10', freq=hour_resample))


plot_pred(7075, time)
plt.savefig('Model_prediction_dropout.png')


# Testing

Toronto went into first stage of lockdown from 23 March 2020 to 23 June 2020.

Toronto went into Stage 2 from 24 June 2020 to 30 July 2020.

Toronto went into Stage 3 from 31 July 2020.

Toronto was placed under lockdown on 23 November 2020.

Toronto had a province-wide shutdown from 26 December 2020.

Second state of emergency in January 2021 and stay-at-home orders.

Stay-at-home orders lifted on March 8, 2021. 

April 3, 2021, entered a shutdown coupled with a stay-at-home order that lasted until June 2, 2021